# Deconstructing the Fitbit Sleep Score

In this project I will use different Machine Learning models in order to get a better understanding of the Fitbit Sleep Score. For those people who have a Fitbit, you've probably been wondering how exactly Fitbit comes up with your sleep score. Sometimes you sleep for shorter periods of time with similar amounts of REM and deep sleep but still get a better sleep score. Other times you have rather low amounts of REM and deep sleep but a better score than a night of higher amounts of those. What's the secret behind this?
That's precisely what I will answer throughout this project.

In [1]:
# Import all relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Read the data
url = 'https://raw.githubusercontent.com/srijp/Fitbit-Sleep-Score/master/Fitbit_Sleep_JB_041219_010720.csv'
sleep_data = pd.read_csv(url)

In [3]:
sleep_data.head()

,Start Time,End Time,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Minutes REM Sleep,Minutes Light Sleep,Minutes Deep Sleep,overall_score
0,30/6/20 21:57,1/7/20 5:59,402,79,40,481,32,282,88,71.0
1,29/6/20 21:35,30/6/20 6:02,444,63,36,507,51,332,61,78.0
2,28/6/20 22:01,29/6/20 6:01,420,60,36,480,37,335,48,78.0
3,27/6/20 22:05,28/6/20 9:27,567,115,51,682,83,390,94,75.0
4,26/6/20 21:40,27/6/20 7:35,495,100,35,595,75,335,85,78.0


In [4]:
# Drop the last row as it doesn't have any sleep score data
sleep_data.dropna(subset=['overall_score'], inplace=True)

For now I will focus on the columns from Minutes Asleep to Minutes Deep Sleep as the features and the overall_score as the label as that most closely resembles the data that the Fitbit App provides to its users. The Number of Awakenings column seems interesting but isn't provided in the app either so I'll drop that one for now as well.

In [5]:
# Obtain column names for features
feats = sleep_data.columns[2:9]

X = sleep_data[feats].astype(float)
X.drop('Number of Awakenings', axis=1, inplace=True)
y = sleep_data['overall_score']

In [6]:
# Split data into training and validation set
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
# Remember: because now I'm looking at a Random Forest Regressor, scaling is not needed

In [7]:
# To obtain a baseline model let's run a Random Forest Regressor with its default settings
rf_base = RandomForestRegressor(random_state=42)
rf_base.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [8]:
# Define a function for scoring the model and returning its accuracy
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    score = model.score(test_features, test_labels)
    print('Model Performance')
    print('Average Error: {:0.4f}.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('Score = {:0.4f}.'.format(score))
    return accuracy

In [9]:
evaluate(rf_base, X_valid, y_valid)

Model Performance
Average Error: 2.6167.
Accuracy = 96.11%.
Score = 0.7234.


96.110849651543

In [38]:
def get_feature_importances(df, model):
    feature_list = list(df.columns)
    importances = list(model.feature_importances_)

    # List of tuples with variable and importance ans subsequent sorting
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse=True)

    # Print out features and corresponding importances
    #[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]
    return feature_importances

In [39]:
feature_importances=get_feature_importances(X, rf_base)

In [40]:
# Define function for converting hours and minutes into minutes
def hours_to_mins(time):
    hour = time[0]
    mins = time[1]
    mins = mins + hour * 60
    return mins

In [41]:
X_train.columns

Index(['Minutes Asleep', 'Minutes Awake', 'Time in Bed', 'Minutes REM Sleep',
       'Minutes Light Sleep', 'Minutes Deep Sleep'],
      dtype='object')

In [42]:
yesterday = [(7,12), (1,20), (8,32), (1,3), (4,45), (1,24)]

In [43]:
# Define function to transform input times
def get_input(times):
    transformed = []
    for time in times:
        transformed.append(hours_to_mins(time))
    transformed = np.array(transformed)
    transformed = transformed.reshape(1, -1)
    return transformed

In [44]:
# Convert last nights sleep score
last_night = get_input(yesterday)
last_night

array([[432,  80, 512,  63, 285,  84]])

In [45]:
# Make a sleep score prediction (the actual sleep score was 77)
rf_base.predict(last_night)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


array([75.8])

In [46]:
# Make prediction for Sasha
sash = [(7,36), (0,42), (8,18), (1,34), (4,31), (1,31)]
sash_last_night = get_input(sash)

In [47]:
rf_base.predict(sash_last_night)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


array([78.83])

In [48]:
# Look at parameters currently in use
pprint(rf_base.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [49]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [50]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [51]:
# View the best hyperparameters
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [52]:
# Create and fit a model with the optimal hyperparameters
best_rf = RandomForestRegressor(n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features='auto', max_depth=90, bootstrap=True, random_state=42)
best_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=90, min_samples_leaf=2, n_estimators=2000,
                      random_state=42)

In [53]:
# View the accuracy and score of the optimal model
evaluate(best_rf, X_valid, y_valid)

Model Performance
Average Error: 2.6152.
Accuracy = 96.11%.
Score = 0.7155.


96.1052016502107

In [54]:
best_rf.predict(last_night)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


array([75.75230536])

In [55]:
# Create list of features with relatively low importance
low_imp = [x[0] for x in feature_importances[3:]]
low_imp
X_train.columns

Index(['Minutes Asleep', 'Minutes Awake', 'Time in Bed', 'Minutes REM Sleep',
       'Minutes Light Sleep', 'Minutes Deep Sleep'],
      dtype='object')

In [56]:
# Drop the low importance features from X_train and X_valid
X_train_reduced = X_train.drop(low_imp, axis=1)
X_valid_reduced = X_valid.drop(low_imp, axis=1)

In [57]:
# Create reduced model with default hyperparameters
rf_reduced = RandomForestRegressor(random_state=42)
rf_reduced.fit(X_train_reduced, y_train)

RandomForestRegressor(random_state=42)

In [58]:
# Evaluate teh reduced model
evaluate(rf_reduced, X_valid_reduced, y_valid)

Model Performance
Average Error: 2.5226.
Accuracy = 96.26%.
Score = 0.7373.


96.25910007634091

In [59]:
get_feature_importances(X_train_reduced, rf_reduced)

[('Minutes Asleep', 0.65), ('Minutes REM Sleep', 0.2), ('Minutes Awake', 0.15)]

In [60]:
# Reduce the last_night array to the reduced features
last_night_reduced = last_night[0][0:4]
last_night_reduced = np.delete(last_night_reduced, 2)
last_night_reduced = last_night_reduced.reshape(1,-1)

In [61]:
# Predict last nights sleep score using the reduced model
rf_reduced.predict(last_night_reduced)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


array([75.72])